In [127]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

## Data inputs

In [128]:
ans = pd.read_csv('../input/answers.csv')
q = pd.read_csv('../input/questions.csv')
prof = pd.read_csv('../input/professionals.csv')
st = pd.read_csv('../input/students.csv')

#### Merging datasets

In [129]:
qa = ans.merge(q, left_on = 'answers_question_id', right_on = 'questions_id',
                        how = 'outer')

In [130]:
qap = qa.merge(prof, left_on = 'answers_author_id', right_on = 'professionals_id',
                    how = 'outer')

In [131]:
qaps = qap.merge(st, left_on = 'questions_author_id', right_on = 'students_id',
                    how = 'outer')

In [132]:
#qaps.head(1).T
#qaps[(qaps.answers_question_id.isnull()) & (~qaps.questions_id.isnull())].shape[0]
if (qaps[(~qaps.answers_question_id.isnull()) & (qaps.questions_id.isnull())].shape[0] > 0): print("Problem with the Questions ID's")

In [133]:
qaps[(~qaps.answers_author_id.isnull()) & (qaps.professionals_id.isnull())].shape[0] # Why???

1017

In [135]:
qaps[(~qaps.questions_author_id.isnull()) & (qaps.students_id.isnull())].shape[0]

413

In [136]:
del qap

In [137]:
cols_remove = ['answers_body', 'questions_title', 'questions_body', 'professionals_headline']

In [138]:
qaps = qaps.drop(cols_remove, axis = 1)

In [139]:
gc.collect()

1330

In [140]:
qaps.head(2).T

,0,1
answers_id,4e5f01128cae4f6d8fd697cec5dca60c,334f6735d31e45589e43da5ae7056e50
answers_author_id,36ff3b3666df400f956f8335cf53e09e,05ab77d4c6a141b999044ebbf5415b0d
answers_question_id,332a511f1569444485cf7a7a556a5e54,7a0d4bc67b1c492fb06fe455b1c07faf
answers_date_added,2016-04-29 19:40:14 UTC+0000,2018-03-08 18:23:36 UTC+0000
questions_id,332a511f1569444485cf7a7a556a5e54,7a0d4bc67b1c492fb06fe455b1c07faf
questions_author_id,8f6f374ffd834d258ab69d376dd998f5,8f6f374ffd834d258ab69d376dd998f5
questions_date_added,2016-04-26 11:14:26 UTC+0000,2016-04-26 10:59:44 UTC+0000
professionals_id,36ff3b3666df400f956f8335cf53e09e,05ab77d4c6a141b999044ebbf5415b0d
professionals_location,"Cleveland, Ohio",Greater New York City Area
professionals_industry,Mental Health Care,Education Management


## Processing data

### Question space

#### Number of answers per question

In [142]:
ans_per_q = ans.groupby('answers_question_id').size() #size() includes NaN values
ans_per_q = pd.DataFrame(ans_per_q.rename('count'))
qaps['ans_per_q'] = pd.merge(qaps, ans_per_q, left_on='questions_id', right_index=True, how='left')['count'].fillna(0).astype(int)

In [143]:
# Sanity check
#qaps[(qaps.ans_per_q > 0) & (qaps.questions_id.isnull())].shape[0]
#qaps[(qaps.ans_per_q > 0) & (qaps.answers_id.isnull())].shape[0]
if (qaps[(qaps.ans_per_q == 0) & (~qaps.answers_id.isnull())].shape[0] > 0): print("There is an error with this metric")

#### First and Last answer per question

In [144]:
first_ans = ans[['answers_question_id', 'answers_date_added']].groupby('answers_question_id').min()
first_ans = first_ans.rename(columns = {'answers_date_added': 'first_ans_per_q'})

last_ans = ans[['answers_question_id', 'answers_date_added']].groupby('answers_question_id').max()
last_ans = last_ans.rename(columns = {'answers_date_added': 'last_ans_per_q'})

qaps['first_ans_per_q'] = pd.merge(qaps, first_ans, 
                                   left_on='questions_id',
                                   right_index=True, how='left')['first_ans_per_q']

qaps['last_ans_per_q'] = pd.merge(qaps, last_ans, 
                                   left_on='questions_id',
                                   right_index=True, how='left')['last_ans_per_q']

In [123]:
#qaps[qaps.ans_per_q == 58].answers_date_added.max()

In [146]:
num_days = (qaps.questions_date_added.max() - qaps.questions_date_added.min()).days
print("Number of days since the first question: ", num_days)

Number of days since the first question:  2683


#### Time to answer a question

In [152]:
qaps['questions_date_added']  = pd.to_datetime(qaps['questions_date_added'], infer_datetime_format=True)
qaps['answers_date_added']  = pd.to_datetime(qaps['answers_date_added'], infer_datetime_format=True)
qaps['professionals_date_joined']  = pd.to_datetime(qaps['professionals_date_joined'], infer_datetime_format=True)
qaps['students_date_joined']  = pd.to_datetime(qaps['students_date_joined'], infer_datetime_format=True)
qaps['first_ans_per_q']  = pd.to_datetime(qaps['first_ans_per_q'], infer_datetime_format=True)
qaps['last_ans_per_q']  = pd.to_datetime(qaps['last_ans_per_q'], infer_datetime_format=True)

In [153]:
qaps['q_answered'] = 0
qaps.loc[(~qaps.answers_id.isnull()) & (~qaps.questions_id.isnull()),'q_answered'] = 1

In [154]:
qaps.loc[qaps.q_answered == 1,'days_to_ans'] = qaps.answers_date_added - qaps.questions_date_added
qaps.loc[qaps.q_answered == 1,'days_first_ans'] = qaps.first_ans_per_q - qaps.questions_date_added

In [155]:
print("Average number of days to answer a question: ", qaps.days_to_ans.mean(skipna = True).days)
print("Average number of days for the first answer: ", qaps.days_first_ans.mean(skipna = True).days)

Average number of days to answer a question:  139
Average number of days for the first answer:  48


In [156]:
qaps.head(2).T

,0,1
answers_id,4e5f01128cae4f6d8fd697cec5dca60c,334f6735d31e45589e43da5ae7056e50
answers_author_id,36ff3b3666df400f956f8335cf53e09e,05ab77d4c6a141b999044ebbf5415b0d
answers_question_id,332a511f1569444485cf7a7a556a5e54,7a0d4bc67b1c492fb06fe455b1c07faf
answers_date_added,2016-04-29 19:40:14,2018-03-08 18:23:36
questions_id,332a511f1569444485cf7a7a556a5e54,7a0d4bc67b1c492fb06fe455b1c07faf
questions_author_id,8f6f374ffd834d258ab69d376dd998f5,8f6f374ffd834d258ab69d376dd998f5
questions_date_added,2016-04-26 11:14:26,2016-04-26 10:59:44
professionals_id,36ff3b3666df400f956f8335cf53e09e,05ab77d4c6a141b999044ebbf5415b0d
professionals_location,"Cleveland, Ohio",Greater New York City Area
professionals_industry,Mental Health Care,Education Management
